In [0]:
df = spark.read.format("csv") \
    .option("header","true") \
    .option("inferSchema","true") \
    .load("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv")

df_sample = df.limit(300)

In [0]:
import time

start = time.time()

df = spark.sql("""
SELECT user_id, COUNT(*) as total_purchases
FROM events_delta
WHERE event_type = 'purchase'
GROUP BY user_id
""")

df.show()

end = time.time()
print("Execution time:", end - start)

+---------+---------------+
|  user_id|total_purchases|
+---------+---------------+
|562785709|              2|
|559882492|              2|
|515126999|              2|
|560127436|             21|
|512860682|              4|
|514738870|              7|
|515025844|              3|
|549256733|              1|
|519393303|              2|
|542076641|              1|
|524566828|             22|
|522415286|              1|
|551524811|              3|
|512370169|              6|
|513998281|              1|
|517096288|              7|
|547289834|              1|
|512972438|              2|
|514141844|              8|
|542769532|              1|
+---------+---------------+
only showing top 20 rows
Execution time: 5.674102067947388


In [0]:
df.explain(True)

== Parsed Logical Plan ==
'Aggregate ['user_id], ['user_id, 'COUNT(1) AS total_purchases#13287]
+- 'Filter ('event_type = purchase)
   +- 'UnresolvedRelation [events_delta], [], false

== Analyzed Logical Plan ==
user_id: int, total_purchases: bigint
Aggregate [user_id#13325], [user_id#13325, count(1) AS total_purchases#13287L]
+- Filter (event_type#13319 = purchase)
   +- SubqueryAlias workspace.default.events_delta
      +- Relation workspace.default.events_delta[event_time#13318,event_type#13319,product_id#13320,category_id#13321L,category_code#13322,brand#13323,price#13324,user_id#13325,user_session#13326] parquet

== Optimized Logical Plan ==
Aggregate [user_id#13325], [user_id#13325, count(1) AS total_purchases#13287L]
+- Project [user_id#13325]
   +- Filter (isnotnull(event_type#13319) AND (event_type#13319 = purchase))
      +- Relation workspace.default.events_delta[event_time#13318,event_type#13319,product_id#13320,category_id#13321L,category_code#13322,brand#13323,price#1332

In [0]:
spark.sql("DESCRIBE DETAIL events_delta").show(truncate=False)

+------+------------------------------------+------------------------------+-----------+--------+-----------------------+-------------------+----------------+-----------------+--------+-----------+-------------------------------------+----------------+----------------+-----------------------------------------+---------------------------------------------------------------+-------------+
|format|id                                  |name                          |description|location|createdAt              |lastModified       |partitionColumns|clusteringColumns|numFiles|sizeInBytes|properties                           |minReaderVersion|minWriterVersion|tableFeatures                            |statistics                                                     |clusterByAuto|
+------+------------------------------------+------------------------------+-----------+--------+-----------------------+-------------------+----------------+-----------------+--------+-----------+-----------------------

# Recreating table with partitioning (to optimize the search)

In [0]:
events_df = spark.table("events_delta")

In [0]:
events_df.write \
    .format("delta") \
    .partitionBy("event_type") \
    .mode("overwrite") \
    .saveAsTable("events_delta_partitioned")

In [0]:
spark.sql("DESCRIBE DETAIL events_delta_partitioned").show(truncate=False)

+------+------------------------------------+------------------------------------------+-----------+--------+-----------------------+-------------------+----------------+-----------------+--------+-----------+------------------------------------------------------------------------------+----------------+----------------+-----------------------------------------+---------------------------------------------------------------+-------------+
|format|id                                  |name                                      |description|location|createdAt              |lastModified       |partitionColumns|clusteringColumns|numFiles|sizeInBytes|properties                                                                    |minReaderVersion|minWriterVersion|tableFeatures                            |statistics                                                     |clusterByAuto|
+------+------------------------------------+------------------------------------------+-----------+--------+-----

In [0]:
spark.sql("""
SELECT * FROM events_delta_partitioned
WHERE event_type = 'purchase'
""").explain(True)

== Parsed Logical Plan ==
'Project [*]
+- 'Filter ('event_type = purchase)
   +- 'UnresolvedRelation [events_delta_partitioned], [], false

== Analyzed Logical Plan ==
event_time: timestamp, event_type: string, product_id: int, category_id: bigint, category_code: string, brand: string, price: double, user_id: int, user_session: string
Project [event_time#14095, event_type#14096, product_id#14097, category_id#14098L, category_code#14099, brand#14100, price#14101, user_id#14102, user_session#14103]
+- Filter (event_type#14096 = purchase)
   +- SubqueryAlias workspace.default.events_delta_partitioned
      +- Relation workspace.default.events_delta_partitioned[event_time#14095,event_type#14096,product_id#14097,category_id#14098L,category_code#14099,brand#14100,price#14101,user_id#14102,user_session#14103] parquet

== Optimized Logical Plan ==
Filter (isnotnull(event_type#14096) AND (event_type#14096 = purchase))
+- Relation workspace.default.events_delta_partitioned[event_time#14095,event